### 데이터 확인 및 불러오기

- 데이터는 구글 드라이브 > 프로젝트 > 중급 프로젝트 > 원본데이터에 있어요.
- 제공된 문서 파일들과 `data_list.csv` 파일에 담긴 메타데이터를 확인합니다.
- 문서 파일을 불러옵니다. `hwp`와 `pdf` 두 가지 포맷에 대응할 수 있어야 해요.
- 유용한 메타데이터가 무엇일지 판단하여 함께 활용해 보세요.

### 문서 청킹

- 청크 크기, 그리고 청크 간 중첩 크기를 잘 설정하여 문서를 청킹합니다.
- (심화) 제안서 포맷을 활용해 의미 단위까지 고려하여 문서를 청킹해 보고 성능을 확인해 보세요.

### 임베딩 생성

- 임베딩 성능과 비용을 고려해 적절한 임베딩 모델을 선정합니다.
- 사용의 편의성과 검색 기능을 고려해 적절한 Vector DB를 선정합니다.

### Retrieval 기능 구현

- 먼저 naive한 retrieval을 구현해 베이스라인으로 삼으면 좋습니다. 베이스라인을 기반으로 점점 더 retrieval 기능을 고도화해 나가면 됩니다.
- 데이터 소스 문서가 하나가 아니라 여러 개이기 때문에, 타깃 문서를 정확히 찾기 위해 메타데이터 필터링을 활용할 수 있습니다. 이때 사용자가 발주 기관이나 사업명 같은 정보를 비슷하지만 정확하지는 않게 입력하는 케이스도 고려하는 게 좋아요.
- (심화) Retrieval과 관련해 다양한 옵션과 기법을 실험하면서 성능을 확인해 보세요.
    - Retrieval을 위한 프롬프트 엔지니어링
    - Top-k 검색에서 k 값 설정
    - 단순 유사도 기반 검색 / MMR(Maximum Marginal Relevance) / Hybrid Search
    - Multi-Query, Re-Ranking 등 심화 기법

### Generation 기능 구현

- 텍스트 생성 능력과 비용을 고려해 적절한 언어 모델을 선정합니다.
- 원하는 답변 양상과 분량에 따라서 temperature, top_p, max tokens 등 텍스트 생성과 관련된 옵션도 적절하게 설정해 주세요.
- 답변 생성을 위한 최적의 프롬프트를 작성합니다.
    - Retrieval 과정에서 찾은 컨텍스트를 충실히 반영해야 합니다.
    - 불필요한 내용이 답변에 포함되지 않도록 합니다.
    - 응답의 톤이나 스타일을 조정합니다.
    - 비용을 고려해 토큰 사용량도 최적화해 보세요.
- RAG 시스템이 대화 맥락을 유지하여 사용자와 대화를 이어 갈 수 있도록 해 주세요.

### 성능 평가

- 답변의 품질을 평가할 때는 다음과 같은 기준을 사용할 수 있습니다.
    - 사용자가 요청한 내용을 단일 문서에서 정확하게 뽑아내 답변하는지
    - 사용자가 여러 문서에 대해 요청한 내용을 잘 종합해서 답변하는지
    - 후속 질문의 맥락을 잘 이해하고 답변하는지
    - 문서에 포함되어 있지 않은 내용에 대해서는 모른다고 답변하는지
- 위와 같은 기준을 어떤 방식으로 평가하고 어떤 지표로 나타낼지 선정해 보세요.
- 주어진 문서 데이터에 맞게 다양한 질문 세트를 준비하여 성능 평가에 활용해 보세요.
    - 질문 예시
        - 국민연금공단이 발주한 이러닝시스템 관련 사업 요구사항을 정리해 줘.
            - 콘텐츠 개발 관리 요구 사항에 대해서 더 자세히 알려 줘.
            - 교육이나 학습 관련해서 다른 기관이 발주한 사업은 없나?
        - 기초과학연구원 극저온시스템 사업 요구에서 AI 기반 예측에 대한 요구사항이 있나?
            - 그럼 모니터링 업무에 대한 요청사항이 있는지 찾아보고 알려 줘.
        - 한국 원자력 연구원에서 선량 평가 시스템 고도화 사업을 발주했는데, 이 사업이 왜 추진되는지 목적을 알려 줘.
        - 고려대학교 차세대 포털 시스템 사업이랑 광주과학기술원의 학사 시스템 기능개선 사업을 비교해 줄래?
            - 고려대학교랑 광주과학기술원 각각 응답 시간에 대한 요구사항이 있나? 문서를 기반으로 정확하게 답변해 줘.
- 품질 좋은 답변도 중요하지만 응답 속도가 너무 느려져서도 안 됩니다.

In [30]:
# Google Drive Mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [31]:
import os

data_dir = '/content/drive/MyDrive/Data' # Updated path
print(f"Listing contents of directory: {data_dir}")
try:
    for item in os.listdir(data_dir):
        print(item)
except FileNotFoundError:
    print(f"Directory not found: {data_dir}")
except Exception as e:
    print(f"An error occurred while listing directory contents: {e}")

Listing contents of directory: /content/drive/MyDrive/Data
files
data_list.csv
data_list.gsheet


# Install Packages

In [32]:
!pip install langchain langchain-openai easyocr pyhwp hwp-extract pymupdf pymupdf4llm faiss-cpu langchain-community

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/pyhwp-0.1b16.dev0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [33]:
# 필수 빌드 도구 & 라이브러리 설치
!apt-get update
!apt-get install -y build-essential libgsf-1-dev libxml2-dev libglib2.0-dev libiconv-hook-dev

# 소스코드 clone
!git clone https://github.com/mete0r/pyhwp.git
%cd pyhwp

# 설치
!python setup.py install

%cd ..

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [34]:
!apt-get update && apt-get install -y libreoffice
!pip install easyocr pymupdf pillow numpy

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

## Load data_list.csv and find NaN values

In [35]:
import pandas as pd
import os

data_dir_path = '/content/drive/MyDrive/Data' # Updated path
csv_path = os.path.join(data_dir_path, 'data_list.csv')

try:
    # Load the CSV file into a pandas DataFrame
    data_list_df = pd.read_csv(csv_path)

    # Display the first few rows of the DataFrame
    print("First 5 rows of data_list.csv:")
    display(data_list_df.head())

    # Check for NaN values in each column
    print("\nChecking for NaN values:")
    nan_counts = data_list_df.isnull().sum()

    # Display the count of NaN values per column
    print("Number of NaN values per column:")
    print(nan_counts)

except FileNotFoundError:
    print(f"Error: The file {csv_path} was not found.")
except Exception as e:
    print(f"An error occurred while reading the CSV file or checking for NaN values: {e}")

First 5 rows of data_list.csv:


,공고 번호,공고 차수,사업명,사업 금액,발주 기관,공개 일자,입찰 참여 시작일,입찰 참여 마감일,사업 요약,파일형식,파일명,텍스트
0,20241001798,0.0,한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보시스템 고도화,130000000.0,한영대학,2024-10-04 13:51:23,NaN,2024-10-15 17:00:00,- 한영대학교 특성화 맞춤형 교육환경 구축을 위해 트랙운영 학사정보시스템을 고도화한...,hwp,한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp,\n \n2024년 특성화 맞춤형 교육환경 구축 – 트랙운영 학사정보시스템 ...
1,20241002912,0.0,2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선,129300000.0,한국연구재단,2024-10-04 15:01:52,2024-10-14 10:00:00,2024-10-16 14:00:00,- 사업 개요: 2024년 대학 산학협력활동 실태조사 시스템(UICC) 기능개선\n...,hwp,한국연구재단_2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선.hwp,\r\n \r\n \r\n \r\n제 안 요 청 서\r\n[ 2024년 대학 ...
2,20240827859,0.0,EIP3.0 고압가스 안전관리 시스템 구축 용역,40000000.0,한국생산기술연구원,2024-08-28 11:31:02,2024-08-29 09:00:00,2024-09-09 10:00:00,- 사업 개요: EIP3.0 고압가스 안전관리 시스템 구축 용역\n- 추진배경: 안...,hwp,한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.hwp,\r\n \r\nEIP3.0 고압가스 안전관리\r\n시스템 구축 용역\...
3,20240430918,0.0,도시계획위원회 통합관리시스템 구축용역,150000000.0,인천광역시,2024-04-18 16:26:32,2024-05-02 10:00:00,2024-05-09 16:00:00,- 사업명: 도시계획위원회 통합관리시스템 구축 용역\n- 용역개요: 도시계획위원회와...,hwp,인천광역시_도시계획위원회 통합관리시스템 구축용역.hwp,\r\n \r\n \r\n도시계획위원회 통합관리시스템 구축\r\n제 안 요 청...
4,20240430896,0.0,봉화군 재난통합관리시스템 고도화 사업(협상)(긴급),900000000.0,경상북도 봉화군,2024-04-18 16:33:28,2024-04-26 09:00:00,2024-04-30 17:00:00,- 사업명: 봉화군 재난통합관리시스템 고도화 사업\n- 사업개요: 공동수급(공동이행...,hwp,경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).hwp,\r\n \r\n \r\n제안요청서\r\n \r\n사 업 명\r\n봉화...



Checking for NaN values:
Number of NaN values per column:
공고 번호        18
공고 차수        18
사업명           0
사업 금액         1
발주 기관         0
공개 일자         0
입찰 참여 시작일    26
입찰 참여 마감일     8
사업 요약         0
파일형식          0
파일명           0
텍스트           0
dtype: int64


# Load and Process Documents (using Langchain)

In [53]:
import os
import sys
import fitz  # PyMuPDF
import easyocr
import numpy as np
from PIL import Image
import subprocess
from langchain.schema import Document
import pyhwp
import pandas as pd
import re
import json # Import json to parse block metadata
import tempfile # Import tempfile for creating temporary files/directories
from datetime import datetime # Import datetime for date parsing

# Define a flag to indicate if `load_hwp_text_blocks` is available
# This should be set to True if the user has defined a working version of this function
LOAD_HWP_TEXT_BLOCKS_AVAILABLE = 'load_hwp_text_blocks' in globals()


def find_executable(executable_name):
    """Finds the full path to an executable within the environment's PATH or script directories."""
    # Check in PATH first
    for path in os.environ["PATH"].split(os.pathsep):
        exe_path = os.path.join(path, executable_name)
        if os.path.isfile(exe_path) and os.access(exe_path, os.X_OK):
            return exe_path

    # Check in common script directories relative to sys.executable
    script_dirs = [
        os.path.join(os.path.dirname(sys.executable), 'bin'), # Linux/macOS
        os.path.join(os.path.dirname(sys.executable), 'Scripts') # Windows
    ]
    for script_dir in script_dirs:
        exe_path = os.path.join(script_dir, executable_name)
        if os.path.isfile(exe_path) and os.access(exe_path, os.X_OK):
            return exe_path

    return None


def clean_text(text: str) -> str:
    """
    Normalize whitespace and remove obvious noise while preserving Hangul.
    - Keep Hangul (U+AC00–U+D7A3), ASCII letters/digits, common punctuation.
    - Avoid aggressive regex that nukes Korean.
    """
    # Normalize line breaks → spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove a few known noise markers (tune cautiously)
    text = text.replace('·', '')  # common dot
    text = text.replace("<표>", "").replace("<그림>", "")

    # Collapse long runs of punctuation but KEEP Korean & numbers
    # Allow: Hangul, ASCII letters, digits, whitespace, basic punctuation
    text = re.sub(r'[^\uAC00-\uD7A3a-zA-Z0-9\s.,:;%()\[\]/\-–—+&~!?\'"°]', ' ', text)
    text = re.sub(r'\s{2,}', ' ', text).strip()
    return text


def _run_and_decode(cmd):
    """Runs a command and decodes output robustly."""
    p = subprocess.run(cmd, capture_output=True)
    out = p.stdout
    err = p.stderr
    if p.returncode != 0:
        return p.returncode, None, err.decode(errors='ignore')
    # Try utf-8 first, fallback to cp949
    try:
        txt = out.decode('utf-8')
    except UnicodeDecodeError:
        txt = out.decode('cp949', errors='ignore')
    return 0, txt, err.decode(errors='ignore')

def _find_hwp_txt():
    """Finds the path to pyhwp's hwp5txt or pyhwp5txt executable."""
    for name in ('pyhwp5txt', 'hwp5txt'):
        path = find_executable(name)
        if path:
            return path
    return None

def extract_text_from_pdf_easyocr(pdf_path, ocr_langs=["ko", "en"]):
    """
    Extracts text from a PDF file page by page using easyOCR with improved fidelity.

    Args:
        pdf_path (str): The path to the PDF file.
        ocr_langs (list, optional): List of languages for OCR. Defaults to ["ko", "en"].

    Returns:
        list: A list of dictionaries, each containing 'text', 'method', 'page', and 'extra'.
    """
    blocks = []
    try:
        # Initialize easyocr reader
        # This might take time on the first run due to model downloads
        reader = easyocr.Reader(ocr_langs)
        pdf_document = fitz.open(pdf_path)

        for page_num in range(pdf_document.page_count):
            page = pdf_document.load_page(page_num)

            # Render at 300 DPI for better OCR fidelity (tables/numbers)
            pixmap = page.get_pixmap(dpi=300)
            img = Image.frombytes("RGB", [pixmap.width, pixmap.height], pixmap.samples)
            img_np = np.array(img)

            # Use easyOCR to read text from the image with paragraph=True
            result = reader.readtext(img_np, detail=0, paragraph=True) # paragraph=True groups lines
            page_text = " ".join(result).strip()

            if page_text:
                blocks.append({"text": clean_text(page_text), "method": "pdf_ocr", "page": page_num + 1, "extra": {}})
                print(f"Extracted text from {pdf_path} page {page_num + 1} using easyOCR.")
            else:
                 print(f"No text extracted from {pdf_path} page {page_num + 1} using easyOCR.")


        pdf_document.close()
    except Exception as e:
        print(f"Error processing PDF {pdf_path} with easyOCR: {e}")
        # Return empty list in case of processing error for this file
        return []

    return blocks


# Integrated HWP Extraction function with fallbacks
def extract_text_from_hwp(hwp_path, hwpx_jar=None, soffice_cmd=None, ocr_langs=["ko","en"]):
    """
    Extracts text from an HWP file using pyhwp with fallbacks to load_hwp_text_blocks or PDF+OCR.

    Args:
        hwp_path (str): The path to the HWP file.
        hwpx_jar (str, optional): Path to the hwp2hwpx.jar file. Defaults to None.
        soffice_cmd (str, optional): Path to the LibreOffice executable. Defaults to None.
        ocr_langs (list, optional): List of languages for OCR. Defaults to ["ko","en"].


    Returns:
        list: A list of dictionaries, each containing 'text', 'method', 'page', and 'extra'.
              Returns an empty list if extraction fails completely.
    """
    blocks = []

    # 1. Try pyhwp (hwp5txt/pyhwp5txt)
    hwp_txt_path = _find_hwp_txt()
    if hwp_txt_path:
        try:
            code, stdout_txt, stderr_txt = _run_and_decode([hwp_txt_path, hwp_path])
            if code == 0 and stdout_txt and stdout_txt.strip():
                text = clean_text(stdout_txt)
                if text:
                    # pyhwp extracts the whole document, so treat as a single block
                    blocks.append({"text": text, "method": "pyhwp", "page": None, "extra": {}})
                    print(f"Extracted text from {hwp_path} using pyhwp.")
                    return blocks # Return immediately if successful
                else:
                     print(f"Warning: pyhwp extracted empty text from {hwp_path}.")
            else:
                print(f"Warning: {hwp_txt_path} failed for {hwp_path}: {stderr_txt}")
        except Exception as e:
            print(f"Warning: Error running pyhwp for {hwp_path}: {e}")
    else:
        print("Warning: pyhwp CLI not found (looked for 'pyhwp5txt'/'hwp5txt').")


    # 2. Fallback: Try load_hwp_text_blocks if available (Handles HWPX and potentially PDF+OCR)
    # This requires the user to have defined a working `load_hwp_text_blocks` function.
    if LOAD_HWP_TEXT_BLOCKS_AVAILABLE:
        print(f"Attempting fallback for {hwp_path} using load_hwp_text_blocks...")
        try:
            # Assuming load_hwp_text_blocks takes path, hwpx_jar, soffice_cmd, ocr_langs and returns blocks
            blocks = load_hwp_text_blocks(hwp_path, hwpx_jar=hwpx_jar, soffice_cmd=soffice_cmd, ocr_langs=ocr_langs)
            if blocks:
                print(f"Extracted text from {hwp_path} using load_hwp_text_blocks.")
                return blocks
            else:
                print(f"Warning: load_hwp_text_blocks returned no blocks for {hwp_path}.")
        except Exception as e:
            print(f"Warning: Error running load_hwp_text_blocks for {hwp_path}: {e}")
    else:
         print("Warning: load_hwp_text_blocks function not available. Skipping this fallback.")


    # 3. Minimal Internal PDF OCR Fallback if soffice is available and load_hwp_text_blocks wasn't used
    # This directly converts HWP to PDF and then uses the defined PDF OCR function.
    soffice_bin = find_executable('soffice') if not soffice_cmd else soffice_cmd
    if soffice_bin:
        print(f"Attempting minimal PDF+OCR fallback for {hwp_path} using soffice...")
        try:
            # Convert HWP→PDF
            with tempfile.TemporaryDirectory() as td:
                output_pdf_path = os.path.join(td, os.path.splitext(os.path.basename(hwp_path))[0] + '.pdf')
                print(f"Converting {hwp_path} to PDF at {output_pdf_path} using soffice...")
                code = subprocess.run([soffice_bin, '--headless', '--convert-to', 'pdf', '--outdir', td, hwp_path],
                                      capture_output=True).returncode
                if code == 0 and os.path.exists(output_pdf_path):
                    print(f"Successfully converted {hwp_path} to PDF.")
                    blocks = extract_text_from_pdf_easyocr(output_pdf_path, ocr_langs=ocr_langs)
                    if blocks:
                        print(f"Extracted text from {hwp_path} via PDF→OCR.")
                        return blocks
                    else:
                         print(f"Warning: PDF→OCR extraction returned no blocks for {hwp_path}.")
                else:
                    print(f"Warning: soffice conversion failed for {hwp_path}. Return code: {code}")
        except Exception as e:
            print(f"Warning: Error during PDF+OCR fallback for {hwp_path}: {e}")
    else:
        print("Warning: soffice command not found. Skipping PDF+OCR fallback.")


    # If all methods failed
    print(f"Error: Failed to extract text from HWP file {hwp_path} after trying all methods.")
    return []

def normalize_metadata(metadata: dict) -> dict:
    """
    Normalizes metadata keys and values, including currency and dates.
    """
    normalized = {}
    for key, value in metadata.items():
        # Normalize key names (e.g., remove spaces, convert to lowercase or snake_case)
        # For now, let's keep the original keys but clean up values
        cleaned_key = key.strip() # Simple cleaning

        # Normalize currency (assuming '사업 금액')
        if cleaned_key == '사업 금액' and pd.notna(value):
            try:
                # Remove non-digit characters except period and handle potential commas
                cleaned_value = str(value).replace(',', '').strip()
                # Convert to float, then to integer if it seems like a whole number
                normalized_value = float(cleaned_value)
                if normalized_value.is_integer():
                    normalized_value = int(normalized_value)
                normalized[cleaned_key] = normalized_value
            except (ValueError, TypeError):
                normalized[cleaned_key] = str(value).strip() # Keep as string if conversion fails
        # Normalize dates (assuming '공개 일자', '입찰 참여 시작일', '입찰 참여 마감일')
        elif cleaned_key in ['공개 일자', '입찰 참여 시작일', '입찰 참여 마감일'] and pd.notna(value):
             try:
                 # Attempt to parse various date formats
                 # Prioritize formats with time, then date only
                 if isinstance(value, str):
                     try:
                         normalized_value = datetime.fromisoformat(value.replace('Z', '+00:00')).isoformat()
                     except ValueError:
                         try:
                             normalized_value = datetime.strptime(value, '%Y-%m-%d %H:%M:%S').isoformat()
                         except ValueError:
                             try:
                                 normalized_value = datetime.strptime(value, '%Y-%m-%d').date().isoformat()
                             except ValueError:
                                  normalized_value = str(value).strip() # Keep as string if parsing fails
                 else:
                      normalized_value = str(value).strip() # Keep as string if not string
                 normalized[cleaned_key] = normalized_value
             except Exception:
                 normalized[cleaned_key] = str(value).strip() # Keep as string on any parsing error

        # Handle other metadata values (clean whitespace)
        elif pd.notna(value):
            normalized[cleaned_key] = str(value).strip()
        else:
             normalized[cleaned_key] = None # Keep NaN as None

    return normalized


# Initialize easyocr reader here to avoid re-initialization in PDF function
# This might take time on the first run due to model downloads
try:
    # reader = easyocr.Reader(["ko", "en"]) # Initialize with required languages
     pass # Initialize inside function to avoid global variable issues and allow language specification
except Exception as e:
    print(f"Error initializing easyocr reader: {e}")
    # Handle this error appropriately, perhaps by exiting or setting a flag


data_dir = '/content/drive/MyDrive/Data/files' # Updated path
csv_path = '/content/drive/MyDrive/Data/data_list.csv' # Path to data_list.csv

try:
    # Load data_list.csv into a pandas DataFrame
    data_list_df = pd.read_csv(csv_path)
except FileNotFoundError:
    print(f"Error: data_list.csv not found at {csv_path}")
    data_list_df = pd.DataFrame() # Create an empty DataFrame if file not found
except Exception as e:
    print(f"An error occurred while loading data_list.csv: {e}")
    data_list_df = pd.DataFrame() # Create an empty DataFrame on error


processed_documents = []
failed_files = [] # List to store files that failed processing

# Specify the files to process - based on previous examples and user request for top 4 HWP and top 1 PDF
# This assumes the order in data_list_df reflects the "top" files the user wants to process.
if not data_list_df.empty:
    hwp_files_to_process = data_list_df[data_list_df['파일형식'] == 'hwp']['파일명'].tolist()[:4]
    pdf_files_to_process = data_list_df[data_list_df['파일형식'] == 'pdf']['파일명'].tolist()[:1]
    files_to_process = hwp_files_to_process + pdf_files_to_process
else:
    files_to_process = []
    print("Cannot determine files to process as data_list.csv was not loaded.")


print(f"Attempting to process specified files from directory: {data_dir}")
print(f"Files selected for processing: {files_to_process}")

if not os.path.isdir(data_dir):
    print(f"Error: Directory not found at {data_dir}")
else:
    for filename in files_to_process:
        file_path = os.path.join(data_dir, filename)
        if os.path.isfile(file_path):
            # Get metadata for the current file and normalize it
            file_metadata = data_list_df[data_list_df['파일명'] == filename].iloc[0].to_dict() if filename in data_list_df['파일명'].values else {}
            normalized_metadata = normalize_metadata(file_metadata)

            extracted_blocks = []
            if filename.lower().endswith('.pdf'):
                try:
                    # Use the updated PDF extraction that returns blocks
                    extracted_blocks = extract_text_from_pdf_easyocr(file_path, ocr_langs=["ko", "en"]) # Specify languages
                except Exception as e:
                    print(f"Failed to process PDF {filename} with easyOCR: {e}")
                    failed_files.append(filename)

            elif filename.lower().endswith('.hwp'):
                try:
                    # Use the updated HWP extraction that returns blocks with fallbacks
                    extracted_blocks = extract_text_from_hwp(file_path, soffice_cmd=find_executable('soffice'), ocr_langs=["ko", "en"]) # Specify soffice and languages
                except Exception as e:
                    print(f"Failed to process HWP {filename}: {e}") # Update message
                    failed_files.append(filename)
            else:
                print(f"Skipping unsupported file type: {filename}")
                failed_files.append(filename) # Mark as failed due to unsupported type

            if extracted_blocks:
                for block in extracted_blocks:
                     # Add normalized metadata to the Document object
                    doc_metadata = {
                        "source": file_path,
                        "method": block.get("method", "unknown"),
                        "page": block.get("page", None),
                        **normalized_metadata # Merge normalized file metadata
                    }
                    processed_documents.append(Document(page_content=block["text"], metadata=doc_metadata))
                print(f"Successfully processed file: {filename}")
            else:
                print(f"No text blocks extracted from file: {filename}")
                if filename not in failed_files: # Avoid double-adding if already marked failed
                    failed_files.append(filename)


        else:
            print(f"Warning: Specified file not found: {filename}")
            failed_files.append(filename)


print("\nDocument processing complete.")
print(f"Total documents processed: {len(processed_documents)}")
if failed_files:
    print(f"Failed to process the following files: {failed_files}")


# Display one HWP and one PDF example with metadata if available
print("\nExamples of processed documents with metadata:")
hwp_example = next((doc for doc in processed_documents if doc.metadata.get('파일형식') == 'hwp'), None)
pdf_example = next((doc for doc in processed_documents if doc.metadata.get('파일형식') == 'pdf'), None)

if hwp_example:
    print("--- HWP Document Example ---")
    print(f"Content snippet: {hwp_example.page_content[:500]}...")
    print(f"Metadata: {hwp_example.metadata}")
else:
    print("No HWP documents were successfully processed with metadata.")

if pdf_example:
    print("\n--- PDF Document Example ---")
    print(f"Content snippet: {pdf_example.page_content[:500]}...")
    print(f"Metadata: {pdf_example.metadata}")
else:
    print("\nNo PDF documents were successfully processed with metadata.")

Attempting to process specified files from directory: /content/drive/MyDrive/Data/files
Files selected for processing: ['한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp', '한국연구재단_2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선.hwp', '한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.hwp', '인천광역시_도시계획위원회 통합관리시스템 구축용역.hwp', '고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf']
Extracted text from /content/drive/MyDrive/Data/files/한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp using pyhwp.
Successfully processed file: 한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp
Extracted text from /content/drive/MyDrive/Data/files/한국연구재단_2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선.hwp using pyhwp.
Successfully processed file: 한국연구재단_2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선.hwp
Extracted text from /content/drive/MyDrive/Data/files/한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.hwp using pyhwp.
Successfully processed file: 한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.hwp
Extracted text from /content/drive/MyDrive/Data/files/인천광역시_도시계획위원회 통합관리시스템 구축용역.hwp using pyhwp.
Successfully processed file

In [54]:
# Inspect the extracted text from HWP files before chunking
print("--- Inspecting Extracted HWP Content ---")
hwp_documents = [doc for doc in processed_documents if doc.metadata.get('파일형식') == 'hwp']

if hwp_documents:
    print(f"Found {len(hwp_documents)} HWP documents.")
    for i, doc in enumerate(hwp_documents):
        print(f"\n--- HWP Document {i+1}: {doc.metadata.get('파일명', 'N/A')} ---")
        print(f"Metadata: {doc.metadata}")
        # Print the full content or a substantial part of it
        print("\nContent:")
        print(doc.page_content[:2000]) # Print up to the first 2000 characters to inspect
        print("...") # Indicate truncated content if any
else:
    print("No HWP documents found in processed_documents.")

print("\n--- Inspection Complete ---")

--- Inspecting Extracted HWP Content ---
Found 4 HWP documents.

--- HWP Document 1: 한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp ---
Metadata: {'source': '/content/drive/MyDrive/Data/files/한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp', 'method': 'pyhwp', 'page': None, '공고 번호': '20241001798', '공고 차수': '0.0', '사업명': '한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보시스템 고도화', '사업 금액': 130000000, '발주 기관': '한영대학', '공개 일자': '2024-10-04T13:51:23', '입찰 참여 시작일': None, '입찰 참여 마감일': '2024-10-15T17:00:00', '사업 요약': '- 한영대학교 특성화 맞춤형 교육환경 구축을 위해 트랙운영 학사정보시스템을 고도화한다.\n- 트랙제도를 도입하여 다양한 진로 선택 기회를 제공하고 산업현장의 경쟁력을 강화한다.\n- 효과적인 교육과정 지원 및 대학 체제 개편에 대한 대응체계를 확립한다.\n- 트랙기반 교육과정 운영 및 관리 체계를 개선하고 교수자-학습자 중심의 교육환경을 조성한다.\n- 학사운영 시스템을 고도화하여 대학 체제 개편에 대한 대응체계를 갖출 수 있다.', '파일형식': 'hwp', '파일명': '한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp', '텍스트': '2024년 특성화 맞춤형 교육환경 구축 – 트랙운영 학사정보시스템 고도화\n  제안요청서\n\n2024. 10.\n   \n \n   \n목   차\nⅠ. 사업 안내          -            1\n 1. 사업개요          -            1\n 2. 추진배경 및 필요성           -          

# Implement Semantic Chunking

In [55]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# While true semantic chunking often requires deeper document structure analysis
# or advanced models, we can use RecursiveCharacterTextSplitter with parameters
# tuned to try and respect some structural elements like paragraphs or headings
# by splitting on various characters including newlines.

# You could potentially add logic here to pre-process text based on HTML structure
# if using hwp5html, or analyze PDF layout for more semantic splits.
# For this step, we'll use RecursiveCharacterTextSplitter with more aggressive
# splitting characters to try and capture semantic breaks.

# Initialize a text splitter with additional separators that might indicate semantic breaks
# The order of separators matters: try splitting on larger semantic units first.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Adjust chunk size as needed
    chunk_overlap=100, # Adjust overlap as needed
    separators=["\n\n", "\n", " ", ""] # Try splitting on paragraphs, then lines, then spaces
)

# Split the processed documents into chunks
# Note: If using hwp5html, the page_content will be HTML, and this splitter
# might not optimally handle HTML tags as semantic separators.
# A dedicated HTML splitter or pre-processing might be needed for better results with HTML.
semantic_chunked_documents = text_splitter.split_documents(processed_documents)

print(f"Original documents: {len(processed_documents)}")
print(f"Semantic chunked documents: {len(semantic_chunked_documents)}")

# Display first few semantic chunked documents
if semantic_chunked_documents:
    print("\nFirst 3 semantic chunked documents example:")
    for i in range(min(3, len(semantic_chunked_documents))):
        print(f"--- Semantic Chunk {i+1} ---")
        print(f"Content snippet: {semantic_chunked_documents[i].page_content[:500]}...")
        print(f"Metadata: {semantic_chunked_documents[i].metadata}")

Original documents: 301
Semantic chunked documents: 407

First 3 semantic chunked documents example:
--- Semantic Chunk 1 ---
Content snippet: 2024. 10. 사 업 명 : 한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보시스템 고도화 사업예산 : 130,000,000원 범위 내 (VAT 포함) 사업기간 : 계약일로부터 3개월 (안정화기간 1개월 포함) 기간 및 일정은 학교 사정과 용역대상자와의 협의에 따라 조정될 수 있음 입찰방법 : 제한경쟁입찰(협상에 의한 계약 체결) 학사제도 제도개편과 연계하여 전공교과목 선택폭을 넓히고, 트랙제 교육과정 참여자에게 다양한 진로선택의 기회를 제공 및 취업문 확대 트랙제 교육과정의 도입 및 운영으로 산업현장의 경쟁력 강화 산업체 수요 맞춤 교육과정 운영 및 활성화로 교육과정 내실화 기업수요 연계 확대로 산업체 및 지역사회 현장실무형 인재 양성 트랙기반 교육과정의 운영 및 관리 체계를 효과적으로 지원 교수자학습자 중심의 교육환경 조성을 통한 대학 교육의 가치 구현 학사운영 시스템을 통해 대학 체제 개편에 대한 대응체계 확립 트랙제도 교과과정 개편 및 표준운영관리를 위한 시스템...
Metadata: {'source': '/content/drive/MyDrive/Data/files/한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp', 'method': 'pyhwp', 'page': None, '공고 번호': '20241001798', '공고 차수': '0.0', '사업명': '한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보시스템 고도화', '사업 금액': 130000000, '발주 기관': '한영대학', '공개 일자': '2024-10-04T13:51:23', '입찰 참여 시작일': None, '입찰 참여 마감일': '2024-10-15T17:00:00', '사업 

In [56]:
# Re-run Embedding Generation and Vector DB (FAISS) Creation
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from google.colab import userdata

try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except userdata.SecretNotFoundError:
    print("Error: OPENAI_API_KEY not found in Colab secrets.")
    print("Please add your OpenAI API key to the Colab secrets manager (🔑 icon on the left) with the name 'OPENAI_API_KEY'.")
    OPENAI_API_KEY = None


if OPENAI_API_KEY and semantic_chunked_documents:
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)
    vectorstore = FAISS.from_documents(semantic_chunked_documents, embeddings)

    print("FAISS vector store created with embeddings from semantic chunks.")
    print(f"Number of documents in vector store: {vectorstore.index.ntotal}")

else:
    print("Skipping vector store creation due to missing API key or no semantic chunks.")
    vectorstore = None

FAISS vector store created with embeddings from semantic chunks.
Number of documents in vector store: 407


In [57]:
# Re-run Advanced Retrieval (using Langchain)
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers import MultiQueryRetriever
from langchain.prompts import PromptTemplate
from google.colab import userdata

try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except userdata.SecretNotFoundError:
    print("Error: OPENAI_API_KEY not found in Colab secrets.")
    print("Please add your OpenAI API key to the Colab secrets manager (🔑 icon on the left) with the name 'OPENAI_API_KEY'.")
    OPENAI_API_KEY = None

if OPENAI_API_KEY and vectorstore:
    llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY, temperature=0.3, max_tokens=1500, top_p=0.8)

    template = """다음 문서를 사용하여 질문에 답하세요.
    답변은 가능한 한 정확하고 간결하게 작성하세요.
    제공된 문서에 없는 내용은 언급하지 마세요.
    문서:
    {context}

    질문: {question}

    답변:"""
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

    base_retriever = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 10, "fetch_k": 20}
    )

    multiquery_retriever = MultiQueryRetriever.from_llm(
        retriever=base_retriever,
        llm=llm
    )

    final_retriever = multiquery_retriever

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=final_retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
    )

    print("Advanced RetrievalQA chain created with Multi-Query, MMR, and refined generation parameters/prompt.")

else:
    print("Skipping Advanced RAG chain creation due to missing API key, vector store, or no semantic chunks.")
    qa_chain = None

Advanced RetrievalQA chain created with Multi-Query, MMR, and refined generation parameters/prompt.


In [58]:
import time

def time_rag_query(qa_chain, query):
    """
    Times the execution of a RAG query.

    Args:
        qa_chain: The RAG chain object.
        query (str): The query string.

    Returns:
        tuple: A tuple containing the response dictionary and the execution time in seconds.
    """
    start_time = time.time()
    response = qa_chain.invoke({"query": query})
    end_time = time.time()
    execution_time = end_time - start_time
    return response, execution_time

In [59]:
# Re-run Example Query 1
if qa_chain and processed_documents:
    hwp1_doc = next((doc for doc in processed_documents if '한영대학' in doc.metadata.get('파일명', '')), None)
    if hwp1_doc:
        query = "한영대학교 트랙운영 학사정보시스템 고도화 사업의 추진 배경 및 필요성은 무엇인가요?"
        response, query_time = time_rag_query(qa_chain, query)

        print("Query:")
        print(query)
        print("\nGenerated Response:")
        print(response['result'])
        print(f"\nQuery Time: {query_time:.2f} seconds")
        print("\nSource Documents:")
        for doc in response['source_documents']:
            print(f"--- Source: {doc.metadata.get('source', 'N/A')} (Chunk Index: {doc.metadata.get('chunk_index', 'N/A')}) ---")
            print(doc.page_content[:300] + "...")
    else:
        print("Could not find processed document for 한영대학 to generate query.")
else:
    print("RAG chain or processed documents not initialized.")

Query:
한영대학교 트랙운영 학사정보시스템 고도화 사업의 추진 배경 및 필요성은 무엇인가요?

Generated Response:
한영대학교 트랙운영 학사정보시스템 고도화 사업의 추진 배경 및 필요성은 다음과 같습니다:

1. **교육 환경의 급격한 변화 대응**: 학령인구 감소, 코로나19 팬데믹으로 인한 교육환경 변화, 인공지능 활용 등으로 대학은 급격한 사회 및 기술 변화에 직면하고 있음.
   
2. **분산된 시스템 및 데이터의 통합**: 정보화 요구사항이 증가하나, 노후화된 학사 시스템으로 인해 업무마다 분산된 정보화 추진이 필요함.

3. **데이터 기반 대학경영 지원 개선**: 분절된 정보 시스템 간 정보 연계 미흡 및 정보화 표준 부재로 인해 경영현황 파악 및 의사결정에 한계가 발생하고 있음.

4. **사용자 정보서비스 접근성 개선**: 교내 구성원의 정보 탐색 지원을 위한 포털 서비스가 제공되고 있으나, 정보 접근성 및 유용성 개선 요구가 큼.

5. **대학 경쟁력 강화 및 전략 목표 달성 지원**: 노후화된 시스템의 차세대 구축을 통해 정보 서비스 품질을 강화하고 대학 교육 시스템의 경쟁력을 확보하고자 함.

Query Time: 8.48 seconds

Source Documents:
--- Source: /content/drive/MyDrive/Data/files/한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp (Chunk Index: N/A) ---
2024. 10. 사 업 명 : 한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보시스템 고도화 사업예산 : 130,000,000원 범위 내 (VAT 포함) 사업기간 : 계약일로부터 3개월 (안정화기간 1개월 포함) 기간 및 일정은 학교 사정과 용역대상자와의 협의에 따라 조정될 수 있음 입찰방법 : 제한경쟁입찰(협상에 의한 계약 체결) 학사제도 제도개편과 연계하여 전공교과목 선택폭을 넓히고, 트랙제 교육과정 참여자에게 다양한 진로선택의 기회를 제

In [60]:
# Re-run Example Query 2
if qa_chain and processed_documents:
    hwp2_doc = next((doc for doc in processed_documents if '한국연구재단' in doc.metadata.get('파일명', '')), None)
    if hwp2_doc:
        query = "한국연구재단의 대학산학협력활동 실태조사 시스템 기능개선 사업의 목표는 무엇인가요?"
        response, query_time = time_rag_query(qa_chain, query)

        print("Query:")
        print(query)
        print("\nGenerated Response:")
        print(response['result'])
        print(f"\nQuery Time: {query_time:.2f} seconds")
        print("\nSource Documents:")
        for doc in response['source_documents']:
            print(f"--- Source: {doc.metadata.get('source', 'N/A')} (Chunk Index: {doc.metadata.get('chunk_index', 'N/A')}) ---")
            print(doc.page_content[:300] + "...")
    else:
        print("Could not find processed document for 한국연구재단 to generate query.")
else:
    print("RAG chain or processed documents not initialized.")

Query:
한국연구재단의 대학산학협력활동 실태조사 시스템 기능개선 사업의 목표는 무엇인가요?

Generated Response:
한국연구재단의 대학산학협력활동 실태조사 시스템 기능개선 사업의 목표는 UICC 시스템 기능개선을 통해 이용 편의성을 개선하고, UICC 시스템 운영 지원을 통해 안정적으로 실태조사를 추진하는 것입니다.

Query Time: 3.69 seconds

Source Documents:
--- Source: /content/drive/MyDrive/Data/files/한국연구재단_2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선.hwp (Chunk Index: N/A) ---
2024. 10. 1. 사업명: 2024년 대학 산학협력활동 실태조사 시스템(UICC) 기능개선 2. 추진배경 및 필요성 한국연구재단법 제5조, 산업교육진흥 및 산학연협력촉진에 관한 법률 제39조의2 및 제43조에 따라 실시하는 대학 산학협력활동 실태조사를 안정적으로 운영 필요 항목 지침 변경사항 등 변화에 신속하고 정확하게 대응하며, 기능개선에 대한 사용자 요구를 반영하여 사용자 편의성 강화 필요 UICC의 효율적인 관리와 운영을 위한 지원 체계 및 각종 기관 요구사항에 대응 필요 3. 주관기관 : 한국연구재단 4. 사업기간 및 ...
--- Source: /content/drive/MyDrive/Data/files/고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf (Chunk Index: N/A) ---
콤W 보롬 고려대학교 KOREA UHIVERsITT 1 서식 15 ] 참여인력 현항표 분야별 소속기관 성명 직급 근무경력 소속 최종학교 참여울(%) 담당업무 사업책임자 부문 부문 부문 부문 부문 부문 부문 부문 부문 부문 X 사업책임자(Project Manager)는 반드시 투입울 10096(전담) 가능자로 인선하여 기재 X 각 부문별 소계 및 전체 합계흘 기술 - 268-...
--- Source: /content/drive/MyDrive/

In [61]:
# Re-run Example Query 3
if qa_chain and processed_documents:
    hwp3_doc = next((doc for doc in processed_documents if '한국생산기술연구원' in doc.metadata.get('파일명', '')), None)
    if hwp3_doc:
        query = "한국생산기술연구원의 EIP3.0 고압가스 안전관리 시스템 구축 용역의 사업 개요를 설명해 주세요."
        response, query_time = time_rag_query(qa_chain, query)

        print("Query:")
        print(query)
        print("\nGenerated Response:")
        print(response['result'])
        print(f"\nQuery Time: {query_time:.2f} seconds")
        print("\nSource Documents:")
        for doc in response['source_documents']:
            print(f"--- Source: {doc.metadata.get('source', 'N/A')} (Chunk Index: {doc.metadata.get('chunk_index', 'N/A')}) ---")
            print(doc.page_content[:300] + "...")
    else:
        print("Could not find processed document for 한국생산기술연구원 to generate query.")
else:
    print("RAG chain or processed documents not initialized.")

Query:
한국생산기술연구원의 EIP3.0 고압가스 안전관리 시스템 구축 용역의 사업 개요를 설명해 주세요.

Generated Response:
EIP3.0 고압가스 안전관리 시스템 구축 용역의 사업 개요는 다음과 같습니다:

1. **사업명**: EIP3.0 고압가스 안전관리 시스템 구축 용역
2. **사업기간**: 계약체결일 ~ 2024.12.27.
3. **사업금액**: 40,000천원(부가세 포함)
4. **계약방법**: 제한경쟁입찰 / 협상에 의한 계약
5. **사업범위**:
   - 원내에서 관리하는 화학물질 운영현황의 정보를 제공하는 시스템 구축
   - 고압가스 화학물질의 정보와 판매허가 업체를 관리하기 위한 시스템 구축
   - 고압가스 화학물질의 구매부터 안전성 검토 과정을 진행하기 위한 시스템 고도화
   - 연구실과 법적 준수 사항을 관리하기 위한 시스템 고도화
   - 화학제품 폐기회수를 관리하기 위한 시스템 고도화

이 사업은 고압가스의 안전한 관리를 통해 연구원들의 생명과 신체를 보호하고, 법적 준수를 통해 안전 관리 수준을 법적 기준 이상으로 유지하는 것을 목표로 하고 있습니다.

Query Time: 11.83 seconds

Source Documents:
--- Source: /content/drive/MyDrive/Data/files/한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.hwp (Chunk Index: N/A) ---
연구관리, 기업지원, 재무, 인사, 급여, 구매조달, 지재권 등 19개 업무시스템 운영 연계시스템 (2023.12.31. 기준) 장비 현황 - 운용 중인 전산장비는 총 1,205종이며, 840종을 서비스수준협약(SLA)을 통하여 안정적인 서비스와 신속한 장애 복구 체계 구축 3. 사업 추진방안 가. 추진목표 원내에서 관리하는 화학물질 운영현황의 통계 정보 공유 고압가스 화학물질의 정보와 판매허가 업체를 관리 고압가스 화학물질의 구매부터 안전성 검토 과정을 제공 연구실과 법적 준수

In [62]:
# Re-run Example Query 4
if qa_chain and processed_documents:
    hwp4_doc = next((doc for doc in processed_documents if '인천광역시' in doc.metadata.get('파일명', '')), None)
    if hwp4_doc:
        query = "인천광역시 도시계획위원회 통합관리시스템 구축 용역의 용역 개요는 무엇인가요?"
        response, query_time = time_rag_query(qa_chain, query)

        print("Query:")
        print(query)
        print("\nGenerated Response:")
        print(response['result'])
        print(f"\nQuery Time: {query_time:.2f} seconds")
        print("\nSource Documents:")
        for doc in response['source_documents']:
            print(f"--- Source: {doc.metadata.get('source', 'N/A')} (Chunk Index: {doc.metadata.get('chunk_index', 'N/A')}) ---")
            print(doc.page_content[:300] + "...")
    else:
        print("Could not find processed document for 인천광역시 to generate query.")
else:
    print("RAG chain or processed documents not initialized.")

Query:
인천광역시 도시계획위원회 통합관리시스템 구축 용역의 용역 개요는 무엇인가요?

Generated Response:
인천광역시 도시계획위원회 통합관리시스템 구축 용역의 용역 개요는 다음과 같습니다:
- 용역명: 도시계획위원회 통합관리시스템 구축 용역
- 기간: 착수일로부터 180일
- 사업비: 금150,000,000원 (VAT 포함)
- 계약방법: 제한경쟁입찰(협상에 의한 계약)

Query Time: 5.52 seconds

Source Documents:
--- Source: /content/drive/MyDrive/Data/files/인천광역시_도시계획위원회 통합관리시스템 구축용역.hwp (Chunk Index: N/A) ---
1. 과 업 명: 도시계획위원회 통합관리시스템 구축 용역 2. 용역개요 용 역 명: 도시계획위원회 통합관리시스템 구축 용역 기 간: 착수일로부터 180일 사 업 비: 금150,000,000원 (VAT 포함) 계약방법: 제한경쟁입찰(협상에 의한 계약) 3. 과업배경 및 목적 (수작업에 의한 업무의 복잡성) 여러 단계의 위원회 운영업무를 수작업으로 처리, 업무의 복잡성 발생 심의자료 변경 시 자료의 재배포 등 업무의 중복성 발생 위원회 전 과정의 지원체계 구축 필요 (통합관리시스템 부재로 업무별 개별 처리) 사전준비 단계에서 행정절차는...
--- Source: /content/drive/MyDrive/Data/files/고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf (Chunk Index: N/A) ---
콤W 보롬 고려대학교 KOREA UHIVERsITT 제2전공(이중전공 움합전공 학생설계전공) 복수전공 등 다중전공별로 일정 관리 다중전공의 신청 차수 지망 차수 관리 여러 다전공 신청 가능 학업계획서 등록 다중전공 선발 기준(학기 학점 평점평균 등)올 충족하지 못할 경우 신청물가사유 안내 신청취소포기신청의 경우 취소사유 포기사유릎 관리 다중전공전형 사정요소(성적 면접 학업계획서 등)별 점수름 관리 학과(부) 대

In [63]:
# Re-run Example Query 5
if qa_chain and processed_documents:
    pdf_doc = next((doc for doc in processed_documents if '고려대학교' in doc.metadata.get('파일명', '')), None)
    if pdf_doc:
        query = "고려대학교 차세대 포털·학사 정보시스템 구축 사업의 추진 배경을 설명해주세요."
        response, query_time = time_rag_query(qa_chain, query)

        print("Query:")
        print(query)
        print("\nGenerated Response:")
        print(response['result'])
        print(f"\nQuery Time: {query_time:.2f} seconds")
        print("\nSource Documents:")
        for doc in response['source_documents']:
            print(f"--- Source: {doc.metadata.get('source', 'N/A')} (Chunk Index: {doc.metadata.get('chunk_index', 'N/A')}) ---")
            print(doc.page_content[:300] + "...")
    else:
        print("Could not find processed document for 고려대학교 to generate query.")
else:
    print("RAG chain or processed documents not initialized.")

Query:
고려대학교 차세대 포털·학사 정보시스템 구축 사업의 추진 배경을 설명해주세요.

Generated Response:
고려대학교 차세대 포털·학사 정보시스템 구축 사업의 추진 배경은 다음과 같습니다:

1. **교육 환경의 급격한 변화 대응**: 학령인구 감소, 코로나19 팬데믹으로 인한 교육 환경 변화, 인공지능의 활용 등으로 대학은 급격한 사회 및 기술 변화에 직면하고 있습니다.

2. **분산된 시스템 및 데이터의 통합**: 급변하는 교육 환경 속에서 정보화 요구사항이 증가하고 있으나, 고려대학교는 노후화된 학사 시스템을 기반으로 업무마다 분산된 정보화 추진을 하고 있어 시스템 통합에 대한 요구가 증가하고 있습니다.

3. **데이터 기반 대학 경영 지원 개선**: 분절된 정보 시스템 간의 정보 연계 미흡 및 정보화 표준 부재로 인해 경영 현황 파악 및 의사결정 시 한계가 발생하고 있습니다.

4. **사용자 정보 서비스 접근성 개선**: 교내 구성원의 정보 탐색 지원을 위한 포털 서비스가 제공되고 있으나, 포털의 정보 접근성 및 유용성에 대한 개선 요구가 상당합니다.

5. **대학 경쟁력 강화 및 전략 목표 달성 지원**: 노후화된 시스템의 차세대 구축을 통해 정보 서비스의 품질을 강화하고 대학 교육 시스템의 경쟁력을 확보하고자 합니다.

Query Time: 8.17 seconds

Source Documents:
--- Source: /content/drive/MyDrive/Data/files/고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf (Chunk Index: N/A) ---
국녀 벼LA 고려대학교 NOREA LHIVER5ITY 사업 개요 1. 사업개요 가 사업명: 고려대학교 차세대 포털학사 정보시스템 구축 사업 나: 사업기간: 계약일로부터 24개월 이내 다: 무상유지보수기간 사업종료일로부터 12개월 라: 사업예산 : 71,270,000,000원 (VAT 포함 3년 분할 지급) 2024학년도 약 309 지급 2025학년도 약

# Task
Implement metadata filtering and conversational memory in the RAG chain and update the example queries to demonstrate these features.

## Implement metadata filtering

### Subtask:
Modify the retrieval process to allow filtering documents based on metadata.


**Reasoning**:
Update the RetrievalQA chain creation to include metadata filtering capabilities in the retriever.



In [64]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers import MultiQueryRetriever
from langchain.prompts import PromptTemplate
from google.colab import userdata

try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except userdata.SecretNotFoundError:
    print("Error: OPENAI_API_KEY not found in Colab secrets.")
    print("Please add your OpenAI API key to the Colab secrets manager (🔑 icon on the left) with the name 'OPENAI_API_KEY'.")
    OPENAI_API_KEY = None

if OPENAI_API_KEY and vectorstore:
    llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY, temperature=0.3, max_tokens=1500, top_p=0.8)

    template = """다음 문서를 사용하여 질문에 답하세요.
    답변은 가능한 한 정확하고 간결하게 작성하세요.
    제공된 문서에 없는 내용은 언급하지 마세요.
    문서:
    {context}

    질문: {question}

    답변:"""
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

    # Update the base retriever to accept filters
    # The filter structure will be determined dynamically based on the query in later steps.
    # For now, we just modify the retriever to potentially use a filter.
    # Langchain's FAISS retriever supports filtering by metadata.
    # The filter needs to be a dictionary specifying the key and value to filter on.
    # Example: {"발주 기관": "한영대학"}
    # We will demonstrate filtering in the example queries in the next step.

    # We will create a base retriever that accepts a filter parameter in its get_relevant_documents method
    # However, the standard .as_retriever() method from FAISS doesn't directly expose a filter parameter
    # in this way for simple use with RetrievalQA.
    # A more direct way to integrate filtering with RetrievalQA is to pass the filter
    # in the query to the chain. Let's revert the retriever creation back to how it was,
    # and the filtering logic will be applied when invoking the QA chain later.

    # Reverting to the multi-query retriever without explicit filter in the retriever definition
    base_retriever = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 10, "fetch_k": 20}
    )

    multiquery_retriever = MultiQueryRetriever.from_llm(
        retriever=base_retriever,
        llm=llm
    )

    final_retriever = multiquery_retriever

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=final_retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
    )

    print("RetrievalQA chain created, ready for metadata filtering in query invocation.")

else:
    print("Skipping RAG chain creation due to missing API key or vector store.")
    qa_chain = None

RetrievalQA chain created, ready for metadata filtering in query invocation.


In [65]:
import time

def time_rag_query_with_filter_corrected(qa_chain, query, metadata_filter=None):
    """
    Times the execution of a RAG query with optional metadata filtering,
    correctly applying the filter to the retriever's search_kwargs.

    Args:
        qa_chain: The RAG chain object.
        query (str): The query string.
        metadata_filter (dict, optional): A dictionary specifying metadata filter criteria.
                                          Defaults to None.

    Returns:
        tuple: A tuple containing the response dictionary and the execution time in seconds.
    """
    start_time = time.time()

    # Access the retriever within the QA chain
    # The exact path might vary slightly depending on the chain type and retriever wrapping
    # If using MultiQueryRetriever wrapped around a base retriever, we need to access the base retriever
    if hasattr(qa_chain.retriever, 'retrievers') and isinstance(qa_chain.retriever.retrievers, list):
         # This case might apply if using something like EnsembleRetriever or similar multi-retriever setups
         # For MultiQueryRetriever, the base retriever is usually accessed differently
         base_retriever = qa_chain.retriever.retrievers[0] # Assuming the first retriever is the base one
    elif hasattr(qa_chain.retriever, 'retriever'):
         # This is the case for ContextualCompressionRetriever and likely MultiQueryRetriever
         base_retriever = qa_chain.retriever.retriever
    else:
         # Direct retriever instance
         base_retriever = qa_chain.retriever


    # Temporarily modify the retriever's search_kwargs to include the filter
    original_search_kwargs = base_retriever.search_kwargs
    if metadata_filter:
        # Create a copy of original search_kwargs and add the filter
        modified_search_kwargs = original_search_kwargs.copy()
        modified_search_kwargs["filter"] = metadata_filter
        base_retriever.search_kwargs = modified_search_kwargs
        print(f"Applying filter: {metadata_filter}")
    else:
        # Ensure no filter is applied if metadata_filter is None
        if "filter" in original_search_kwargs:
             modified_search_kwargs = original_search_kwargs.copy()
             del modified_search_kwargs["filter"]
             base_retriever.search_kwargs = modified_search_kwargs
        print("No filter applied.")


    # Invoke the chain with the query
    # The search_kwargs are now set on the retriever itself
    response = qa_chain.invoke({"query": query})

    # Restore the original search_kwargs on the retriever
    base_retriever.search_kwargs = original_search_kwargs

    end_time = time.time()
    execution_time = end_time - start_time
    return response, execution_time

# Example Query 1 with Metadata Filter (Filter by '발주 기관': '한영대학')
if qa_chain and processed_documents:
    query = "이 사업의 추진 배경 및 필요성은 무엇인가요?"
    metadata_filter = {"발주 기관": "한영대학"}
    print(f"Query: {query}")
    response, query_time = time_rag_query_with_filter_corrected(qa_chain, query, metadata_filter)

    print("\nGenerated Response:")
    print(response['result'])
    print(f"\nQuery Time: {query_time:.2f} seconds")
    print("\nSource Documents:")
    for doc in response['source_documents']:
        print(f"--- Source: {doc.metadata.get('source', 'N/A')} ---")
        print(f"Metadata: {doc.metadata}")
        print(doc.page_content[:300] + "...")
else:
    print("RAG chain or processed documents not initialized.")

print("-" * 50)

# Example Query 2 with Metadata Filter (Filter by '파일형식': 'pdf')
if qa_chain and processed_documents:
    query = "이 사업의 추진 배경을 설명해주세요."
    metadata_filter = {"파일형식": "pdf"}
    print(f"Query: {query}")
    response, query_time = time_rag_query_with_filter_corrected(qa_chain, query, metadata_filter)

    print("\nGenerated Response:")
    print(response['result'])
    print(f"\nQuery Time: {query_time:.2f} seconds")
    print("\nSource Documents:")
    for doc in response['source_documents']:
        print(f"--- Source: {doc.metadata.get('source', 'N/A')} ---")
        print(f"Metadata: {doc.metadata}")
        print(doc.page_content[:300] + "...")
else:
    print("RAG chain or processed documents not initialized.")

print("-" * 50)

# Example Query 3 with Metadata Filter (Filter by '사업명')
if qa_chain and processed_documents:
    # Find an example 사업명 from the processed documents
    example_doc = processed_documents[0] if processed_documents else None
    if example_doc and '사업명' in example_doc.metadata:
        example_사업명 = example_doc.metadata['사업명']
        query = f"'{example_사업명}' 사업의 개요는 무엇인가요?"
        metadata_filter = {"사업명": example_사업명}
        print(f"Query: {query}")
        response, query_time = time_rag_query_with_filter_corrected(qa_chain, query, metadata_filter)

        print("\nGenerated Response:")
        print(response['result'])
        print(f"\nQuery Time: {query_time:.2f} seconds")
        print("\nSource Documents:")
        for doc in response['source_documents']:
            print(f"--- Source: {doc.metadata.get('source', 'N/A')} ---")
            print(f"Metadata: {doc.metadata}")
            print(doc.page_content[:300] + "...")
    else:
         print("Could not find an example document with '사업명' metadata for Query 3.")
else:
    print("RAG chain or processed documents not initialized.")

Query: 이 사업의 추진 배경 및 필요성은 무엇인가요?
Applying filter: {'발주 기관': '한영대학'}

Generated Response:
제공된 문서에는 이 사업의 추진 배경 및 필요성에 대한 구체적인 내용이 포함되어 있지 않습니다. 따라서 해당 질문에 대한 답변을 제공할 수 없습니다.

Query Time: 3.09 seconds

Source Documents:
--- Source: /content/drive/MyDrive/Data/files/한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp ---
Metadata: {'source': '/content/drive/MyDrive/Data/files/한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp', 'method': 'pyhwp', 'page': None, '공고 번호': '20241001798', '공고 차수': '0.0', '사업명': '한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보시스템 고도화', '사업 금액': 130000000, '발주 기관': '한영대학', '공개 일자': '2024-10-04T13:51:23', '입찰 참여 시작일': None, '입찰 참여 마감일': '2024-10-15T17:00:00', '사업 요약': '- 한영대학교 특성화 맞춤형 교육환경 구축을 위해 트랙운영 학사정보시스템을 고도화한다.\n- 트랙제도를 도입하여 다양한 진로 선택 기회를 제공하고 산업현장의 경쟁력을 강화한다.\n- 효과적인 교육과정 지원 및 대학 체제 개편에 대한 대응체계를 확립한다.\n- 트랙기반 교육과정 운영 및 관리 체계를 개선하고 교수자-학습자 중심의 교육환경을 조성한다.\n- 학사운영 시스템을 고도화하여 대학 체제 개편에 대한 대응체계를 갖출 수 있다.', '파일형식': 'hwp', '파일명': '한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp', '텍스트': '2024년 특성화

## Add conversational memory

### Subtask:
Integrate conversational memory into the RAG chain to maintain context across turns.


**Reasoning**:
Integrate conversational memory into the RAG chain to maintain context across turns.



In [66]:
from langchain.chains import RetrievalQA
# Import the necessary components for conversational memory
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain.retrievers import MultiQueryRetriever
from langchain.prompts import PromptTemplate
from google.colab import userdata

try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except userdata.SecretNotFoundError:
    print("Error: OPENAI_API_KEY not found in Colab secrets.")
    print("Please add your OpenAI API key to the Colab secrets manager (🔑 icon on the left) with the name 'OPENAI_API_KEY'.")
    OPENAI_API_KEY = None # Set to None to prevent further errors

if OPENAI_API_KEY and vectorstore:
    # Initialize the ChatOpenAI model for generation (will also be used for Multi-Query)
    llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY, temperature=0.3, max_tokens=1500, top_p=0.8)

    # Define a refined prompt template that includes chat history
    # This template guides the language model on how to use the retrieved context and the conversation history
    template = """다음 채팅 기록과 문서를 사용하여 질문에 답하세요.
    채팅 기록:
    {chat_history}

    문서:
    {context}

    질문: {question}

    답변:"""
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

    # Base Retriever with higher k and MMR
    base_retriever = vectorstore.as_retriever(
        search_type="mmr", # Use MMR search
        search_kwargs={"k": 10, "fetch_k": 20} # Increased k and fetch_k
    )

    # Implement Multi-Query Retriever
    # The multi-query retriever itself does not inherently handle conversational context
    # It generates multiple queries based on the *current* query.
    # Conversational memory needs to be integrated at the chain level.
    multiquery_retriever = MultiQueryRetriever.from_llm(
        retriever=base_retriever,
        llm=llm
    )

    final_retriever = multiquery_retriever

    # Initialize ConversationBufferMemory to store chat history
    # memory_key='chat_history' corresponds to the chat history variable in the prompt template
    # input_key='question' corresponds to the variable in the chain that will receive the user's query
    memory = ConversationBufferMemory(memory_key="chat_history", input_key="question")


    # Create a RetrievalQA chain with the advanced retriever and refined prompt
    # Add the memory parameter to the chain
    qa_chain_with_memory = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff", # 'stuff' chain type is suitable for including context in the prompt
        retriever=final_retriever, # Use the advanced retriever
        return_source_documents=True,
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT, "memory": memory} # Pass the refined prompt and memory
    )

    # Update the qa_chain variable
    qa_chain = qa_chain_with_memory

    print("Advanced RetrievalQA chain created with Multi-Query, MMR, refined generation parameters/prompt, and conversational memory.")

else:
    print("Skipping RAG chain creation due to missing API key or vector store.")
    qa_chain = None # Set qa_chain to None if creation failed

Advanced RetrievalQA chain created with Multi-Query, MMR, refined generation parameters/prompt, and conversational memory.


**Reasoning**:
Demonstrate the conversational memory by asking follow-up questions that rely on the previous turn's context.



In [67]:
# Function to time RAG query (re-using the previous definition for consistency)
import time

def time_rag_query(qa_chain, query):
    """
    Times the execution of a RAG query.

    Args:
        qa_chain: The RAG chain object.
        query (str): The query string.

    Returns:
        tuple: A tuple containing the response dictionary and the execution time in seconds.
    """
    start_time = time.time()
    response = qa_chain.invoke({"query": query})
    end_time = time.time()
    execution_time = end_time - start_time
    return response, execution_time

# Example Conversation with Conversational Memory
if qa_chain and processed_documents:
    print("--- Starting Conversation with Memory ---")

    # First turn: Ask about a specific project's background (similar to previous examples)
    query1 = "한영대학교 트랙운영 학사정보시스템 고도화 사업의 추진 배경 및 필요성은 무엇인가요?"
    print(f"\nUser Query 1: {query1}")
    response1, time1 = time_rag_query(qa_chain, query1)
    print("\nGenerated Response 1:")
    print(response1['result'])
    print(f"Query 1 Time: {time1:.2f} seconds")
    # Source documents are returned but not explicitly printed here for brevity,
    # as the focus is on the conversational flow.

    print("-" * 30)

    # Second turn: Ask a follow-up question that relies on the previous turn's context
    # This query doesn't mention "한영대학교" or the full project name,
    # relying on the memory to understand "이 사업" (this project).
    query2 = "그 사업의 사업 기간은 얼마나 되나요?"
    print(f"\nUser Query 2: {query2}")
    response2, time2 = time_rag_query(qa_chain, query2)
    print("\nGenerated Response 2:")
    print(response2['result'])
    print(f"Query 2 Time: {time2:.2f} seconds")

    print("-" * 30)

    # Third turn: Another follow-up question
    query3 = "사업 금액은 얼마인가요?"
    print(f"\nUser Query 3: {query3}")
    response3, time3 = time_rag_query(qa_chain, query3)
    print("\nGenerated Response 3:")
    print(response3['result'])
    print(f"Query 3 Time: {time3:.2f} seconds")

    print("--- Conversation with Memory Ended ---")

else:
    print("RAG chain with memory or processed documents not initialized.")

--- Starting Conversation with Memory ---

User Query 1: 한영대학교 트랙운영 학사정보시스템 고도화 사업의 추진 배경 및 필요성은 무엇인가요?

Generated Response 1:
한영대학교 트랙운영 학사정보시스템 고도화 사업의 추진 배경 및 필요성은 다음과 같습니다:

1. **학사제도 개편과 연계**: 전공교과목 선택폭을 넓히고, 트랙제 교육과정 참여자에게 다양한 진로선택의 기회를 제공하여 취업문을 확대하기 위한 필요성이 있습니다.

2. **산업현장 경쟁력 강화**: 트랙제 교육과정의 도입 및 운영을 통해 산업체 수요에 맞춘 교육과정을 운영하고 활성화하여 교육과정의 내실화를 도모하고, 기업 수요에 연계하여 산업체 및 지역사회에서 현장 실무형 인재를 양성할 필요성이 있습니다.

3. **효과적인 지원 체계 구축**: 트랙 기반 교육과정의 운영 및 관리 체계를 효과적으로 지원하기 위해 교수자와 학습자 중심의 교육환경을 조성하고, 대학 교육의 가치를 구현할 필요가 있습니다.

4. **학사운영 시스템의 고도화**: 대학 체제 개편에 대한 대응체계를 확립하고, 트랙제도 교과과정 개편 및 표준 운영 관리를 위한 시스템 구현이 필요합니다. 이는 교육과정, 수강신청, 성적, 학적 관리 등 다양한 기능을 포함해야 합니다.

5. **확장성과 유연성 고려**: 시스템의 확장성과 유연성을 고려하여 유지보수가 용이하도록 설계해야 하며, 다양한 사용자(교수, 직원, 학생)의 정보 접근성과 사용 편의성을 고려해야 합니다.

이러한 배경과 필요성은 한영대학교의 교육환경 변화에 효과적으로 대응하고, 학생들에게 더 나은 교육 기회를 제공하기 위한 기반을 마련하는 데 중점을 두고 있습니다.
Query 1 Time: 7.38 seconds
------------------------------

User Query 2: 그 사업의 사업 기간은 얼마나 되나요?

Generated Response 2:
한영대학교 트랙운영 학사정보시스템 고도화 